![Image](https://miro.medium.com/v2/resize%3Afit%3A1400/1%2AMI9WDgzoOGAH4bOnAwBKEw.jpeg)

![Image](https://engineering.fb.com/wp-content/uploads/2017/03/GOcmDQEFmV52jukHAAAAAAAqO6pvbj0JAAAB.jpg)

![Image](https://assets.zilliz.com/vector_db_integration_107439d031.png)

![Image](https://miro.medium.com/1%2AKX54wUL3_JPplPiqGPRngw.png)



---

# 📌 Multi-Utility Chatbot with RAG as a Tool (Improved & Contextualized)

## 🧠 Core Architectural Insight

**Traditional RAG Chain:**
`Ingest → Retrieve → Answer`

**This Video’s Novelty:**

> Treat RAG **as an external Tool** — exactly like a Calculator or Stock Price lookup.
> The **LLM decides** whether to call the RAG tool based on the question.

📌 This creates a **dynamic decision boundary**:

* If the question is general (“What is supervised learning?”) → LLM answers from its internal knowledge.
* If the question refers to the uploaded document (“What does the PDF say about supervised learning?”) → LLM triggers the RAG tool.

---

## 🚀 Full Workflow (High-Level)

### Two Phases

```
┌────────────────────────────┐
│ Phase 1: Ingestion (Setup) │
└────────────────────────────┘
        ↓
 Ingest Document → Chunk → Embed → Store

┌───────────────────────────────────┐
│ Phase 2: Retrieval Execution (Run) │
└───────────────────────────────────┘
        ↓
Query → Chat Node → (Optional) Tool Call → LLM Answer
```

---

# 📁 PHASE 1 — Document Ingestion & Vector Store Setup

This code *must run once per new document*.

### 1) Load & Parse PDF

```python
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("intro_to_ml.pdf")
raw_pages = loader.load()  # Metadata + raw page text
```

### 2) Split Into Chunks

```python
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
documents = splitter.split_documents(raw_pages)
```

* **chunk_size** ensures each piece fits within LLM context windows.
* **chunk_overlap** preserves continuity across splits.

### 3) Embeddings

```python
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
```

* Use a **fixed, pinned embedding model** for deterministic behavior.

### 4) Store in FAISS

```python
from langchain_community.vectorstores import FAISS

faiss_index = FAISS.from_documents(documents, embeddings)
retriever = faiss_index.as_retriever(search_kwargs={"k": 4})
```

* **FAISS** is local, high-performance, and vector-based.
* `k=4` retrieves the top 4 semantically closest chunks.

---

# 🛠 PHASE 2 — RAG as a Tool

## 🔧 Define the RAG Tool

```python
from langchain_core.tools import tool

@tool(response_format="content_and_artifact")
def retrieve_information(query: str):
    """
    Given a query, return the top-k document chunks relevant to the query.
    """
    docs = retriever.invoke(query)  # Vector search

    context = "\n\n".join(
        f"Source: {doc.metadata}\nContent: {doc.page_content}"
        for doc in docs
    )

    return context, docs
```

### What This Tool Does

1. Transforms query → vector
2. Performs FAISS search
3. Formats text for LLM consumption
4. Outputs:

   * `context` (for grounding the answer)
   * `docs` (traceable source chunks)

---

# 🧠 SYSTEM INTEGRATION — LangGraph

## Bind the RAG Tool

```python
from langgraph.prebuilt import ToolNode, tools_condition

tools = [retrieve_information, calculator, get_stock_price]
llm_with_tools = llm.bind_tools(tools)
```

## Define Graph

```python
builder = StateGraph(State)

builder.add_node("chat", chat_node)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chat")
builder.add_conditional_edges("chat", tools_condition)
builder.add_edge("tools", "chat")

app = builder.compile()
```

* **chat_node**: initial conversational agent
* **tools_condition**: logic deciding whether to call a tool
* **ToolNode**: encapsulates RAG, Calculator, and Stock APIs

---

# 🧠 DETAILED SEQUENCE DIAGRAM (Concept)

```mermaid
sequenceDiagram
    participant User
    participant Chat as Chat Node
    participant Tool as RAG Tool
    participant FAISS

    User->>Chat: Query
    Chat-->>Chat: Decide (LLM internal logic)
    alt Needs RAG
        Chat->>Tool: retrieve_information(query)
        Tool->>FAISS: Vector search
        FAISS-->>Tool: Top K contexts
        Tool-->>Chat: context + docs
        Chat-->>User: Final answer (grounded)
    else No RAG needed
        Chat-->>User: Direct answer
    end
```

---

# 📌 COMMON QUESTIONS — With Answers

### **Q: What exactly decides if the RAG tool is used?**

**A:**
An LLM classifier/agent logic inside the `chat_node` determines intent. It triggers `retrieve_information` only when the query references the uploaded document or demands factual grounding.

---

### **Q: Why FAISS instead of a cloud vector database?**

**A:**

* FAISS is **local**, fast, and private.
* No external costs or network latency.
* Replaceable with Pinecone/Weaviate if horizontal scalability is required.

---

### **Q: What part ensures answers don’t hallucinate?**

**A:**

* For RAG queries, the tool returns **grounded text chunks**.
* The LLM conditions answers on this context, reducing hallucination.

---

# 🚧 EDGE CASES & ADVERSARIAL CONDITIONS

| Condition                   | Behavior             | Mitigation                                                 |
| --------------------------- | -------------------- | ---------------------------------------------------------- |
| Query unrelated to document | LLM answers directly | Ensure agent doesn’t call RAG tool unnecessarily           |
| Very large documents        | FAISS slows down     | Pre-shard per chapter or use HNSW / optimized vector store |
| Overlapping semantics       | Irrelevant chunks    | Improve chunking & tuning of embeddings                    |
| Ambiguous user intent       | Wrong tool triggered | Add explicit intent classification layer                   |
| Source citation             | User wants source    | Include metadata in responses                              |

---

## 📌 ADDITIONAL OPTIMIZATIONS (When Scaling)

➤ **Persistent Vector Store:** Serialize FAISS to disk with checkpointing.
➤ **Incremental Ingestion:** New docs update vectors without full rebuild.
➤ **Semantic Filtering:** Use query classification to bypass RAG for simple questions.
➤ **Multi-Document Support:** Index multiple PDFs and route queries to correct namespace.

---

## 📊 PERFORMANCE & SLA NOTES

* **Latency:** RAG adds ~7–10 seconds due to vector search + context processing.
* **Throughput:** FAISS local is faster than HTTP database calls.
* **SLO:** 95% of queries return within 12s.
* **Breach Handling:** If RAG fails, fallback to internal LLM answer with “incomplete context”.

---

